In [ ]:
# 1) Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, precision_recall_fscore_support
from xgboost import XGBClassifier
import shap
from lime.lime_tabular import LimeTabularExplainer
import warnings
warnings.filterwarnings("ignore")

# 2) Load dataset 
DATA_PATH = "/mnt/data/credit_risk_dataset.csv"   
assert os.path.exists(DATA_PATH), f"File not found: {DATA_PATH}"
df = pd.read_csv(DATA_PATH)
print("Loaded dataset shape:", df.shape)
df.head()

# 3) Basic preprocessing: create derived features, drop ID, target separation
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'])
df['credit_age_years'] = (pd.Timestamp.today() - df['earliest_cr_line']).dt.days / 365.25

# Drop loan_id (unique id) and earliest_cr_line (we derived credit_age_years)
if 'loan_id' in df.columns:
    df = df.drop(columns=['loan_id'])
if 'earliest_cr_line' in df.columns:
    df = df.drop(columns=['earliest_cr_line'])

# Target
TARGET = 'target_default'
assert TARGET in df.columns, f"Target column '{TARGET}' not found."

# Quick check: convert boolean-like columns if any
# (none expected, but keep safe)
for c in df.select_dtypes(include=['bool']).columns:
    df[c] = df[c].astype(int)

# 4) Train/test split
X = df.drop(columns=[TARGET])
y = df[TARGET].astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.20,
                                                    stratify=y,
                                                    random_state=42)
print("Train shape:", X_train.shape, "Test shape:", X_test.shape, "Default rate:", y.mean())

# 5) Build preprocessing pipeline
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()

print("Numeric cols:", num_cols)
print("Categorical cols:", cat_cols)

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
], remainder='drop')

# 6) Full pipeline with XGBoost classifier
xgb = XGBClassifier(n_estimators=200, learning_rate=0.05, max_depth=6,
                    use_label_encoder=False, eval_metric='logloss',
                    random_state=42, n_jobs= -1)

pipeline = Pipeline([
    ('preproc', preprocessor),
    ('clf', xgb)
])

# 7) Fit the pipeline (quick baseline)
pipeline.fit(X_train, y_train)
print("Baseline training complete.")

# 8) Predictions and evaluation
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:,1]
auc = roc_auc_score(y_test, y_proba)
print(f"AUC on test set: {auc:.4f}")
print("Classification report:")
print(classification_report(y_test, y_pred, digits=4))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)

# Save model
MODEL_PATH = "/mnt/data/xgb_pipeline.joblib"
joblib.dump(pipeline, MODEL_PATH)
print("Saved pipeline to:", MODEL_PATH)

# 9) Extract preprocessor output feature names (works with sklearn >=1.0)
# We'll fit a small helper to get feature names after ColumnTransformer
def get_feature_names(column_transformer):
    """
    Return feature names from all transformers.
    Expects the ColumnTransformer to be fitted.
    """
    output_features = []
    for name, transformer, columns in column_transformer.transformers_:
        if name == 'remainder' and transformer == 'drop':
            continue
        if hasattr(transformer, 'named_steps') and 'ohe' in transformer.named_steps:
            # OneHotEncoder inside pipeline
            ohe = transformer.named_steps['ohe']
            input_cols = columns
            ohe_cols = list(ohe.get_feature_names_out(input_cols))
            output_features.extend(ohe_cols)
        elif isinstance(transformer, OneHotEncoder):
            ohe = transformer
            input_cols = columns
            ohe_cols = list(ohe.get_feature_names_out(input_cols))
            output_features.extend(ohe_cols)
        else:
            # numeric or other pipeline -> pass-through column names (maybe scaled)
            if isinstance(columns, (list, np.ndarray)):
                output_features.extend(list(columns))
            else:
                # if columns is slice / str
                output_features.append(columns)
    return output_features

# fit preprocessor on training data to ensure transformers have been fit
preprocessor_fitted = pipeline.named_steps['preproc']
# If pipeline preproc is already fitted during pipeline.fit, transformers_ exist
feature_names = get_feature_names(preprocessor_fitted)
print("Number of features after preprocessing:", len(feature_names))
# convert X_test transformed to array for use in SHAP/LIME
X_test_preprocessed = preprocessor_fitted.transform(X_test)
if hasattr(X_test_preprocessed, "toarray"):
    X_test_preprocessed = X_test_preprocessed.toarray()

# 10) SHAP: global + local explanations (TreeExplainer for XGBoost)
print("Computing SHAP values (this may take a minute)...")
model_for_shap = pipeline.named_steps['clf']
explainer = shap.TreeExplainer(model_for_shap)
# Use a sample for shap computations to save time
sample_idx = np.random.RandomState(42).choice(X_test_preprocessed.shape[0], size=min(2000, X_test_preprocessed.shape[0]), replace=False)
X_shap_sample = X_test_preprocessed[sample_idx]
shap_values = explainer.shap_values(X_shap_sample)

# SHAP summary plot (global)
plt.figure(figsize=(10,6))
shap.summary_plot(shap_values, X_shap_sample, feature_names=feature_names, show=False)
plt.tight_layout()
SHAP_SUMMARY_PNG = "/mnt/data/shap_summary.png"
plt.savefig(SHAP_SUMMARY_PNG, dpi=150, bbox_inches='tight')
plt.close()
print("Saved SHAP summary plot to:", SHAP_SUMMARY_PNG)

# 11) Save SHAP values for the sample to CSV (top contributions)
# For convenience, compute mean absolute shap per feature (global importance)
mean_abs_shap = np.abs(shap_values).mean(axis=0)
shap_df = pd.DataFrame({'feature': feature_names, 'mean_abs_shap': mean_abs_shap})
shap_df = shap_df.sort_values('mean_abs_shap', ascending=False)
SHAP_CSV = "/mnt/data/shap_feature_importance.csv"
shap_df.to_csv(SHAP_CSV, index=False)
print("Saved SHAP importances to:", SHAP_CSV)

# 12) LIME: local explanations for raw features
# We'll create a wrapper classifier that accepts raw X (pandas row(s)) and returns predict_proba
def predict_proba_raw(raw_X):
    """
    raw_X: 2D numpy array or list of raw feature rows in the original (pre-split) column order.
    Returns: predict_proba of the pipeline for class probabilities.
    """
    if isinstance(raw_X, np.ndarray):
        arr = raw_X
    else:
        arr = np.array(raw_X)
    # If arr is 1D (single instance), reshape
    if arr.ndim == 1:
        arr = arr.reshape(1, -1)
    # Build DataFrame with same columns as X_train
    df_raw = pd.DataFrame(arr, columns=X_train.columns)
    proba = pipeline.predict_proba(df_raw)
    return proba

# Instantiate Lime explainer using training data in raw (original) feature space
X_train_raw = X_train.reset_index(drop=True)
lime_explainer = LimeTabularExplainer(X_train_raw.values,
                                      feature_names=X_train_raw.columns.tolist(),
                                      class_names=['no_default', 'default'],
                                      discretize_continuous=True,
                                      random_state=42)

# 13) Choose three representative cases: clear approval, clear denial, borderline
# We'll pick based on predicted probability extremes and near 0.5

test_df = X_test.reset_index(drop=True)
test_proba = pipeline.predict_proba(test_df)[:,1]
test_pred = pipeline.predict(test_df)

test_results = test_df.copy()
test_results['proba_default'] = test_proba
test_results['pred'] = test_pred
test_results['true'] = y_test.reset_index(drop=True)

# Clear approval: very low predicted prob (<=0.05) and predicted 0
approval_candidates = test_results[test_results['proba_default'] <= 0.05]
approval_idx = approval_candidates.index[0] if len(approval_candidates) > 0 else test_results['proba_default'].idxmin()

# Clear denial: very high predicted prob (>=0.95) and predicted 1
denial_candidates = test_results[test_results['proba_default'] >= 0.95]
denial_idx = denial_candidates.index[0] if len(denial_candidates) > 0 else test_results['proba_default'].idxmax()

# Borderline: nearest to 0.5
borderline_idx = (test_results['proba_default'] - 0.5).abs().idxmin()

selected_indices = [int(approval_idx), int(denial_idx), int(borderline_idx)]
selected_indices, test_results.loc[selected_indices, ['proba_default','pred','true']]

# 14) For each selected case compute SHAP local force values and LIME explanation and save results
CASE_DIR = "/mnt/data/case_explanations"
os.makedirs(CASE_DIR, exist_ok=True)

for idx in selected_indices:
    raw_row = test_df.iloc[idx:idx+1]  # keep as DataFrame
    proba = float(test_results.loc[idx, 'proba_default'])
    pred = int(test_results.loc[idx, 'pred'])
    true = int(test_results.loc[idx, 'true'])
    print(f"\nCase index {idx}  proba_default={proba:.4f} pred={pred} true={true}")
    # SHAP local explanation
    # Need preprocessed version for shap
    x_pre = preprocessor_fitted.transform(raw_row)
    if hasattr(x_pre, "toarray"):
        x_pre = x_pre.toarray()
    shap_local_vals = explainer.shap_values(x_pre)
    # create a DataFrame with shap contributions
    shap_local_df = pd.DataFrame({'feature': feature_names,
                                  'shap_value': shap_local_vals.ravel(),
                                  'abs_shap': np.abs(shap_local_vals).ravel()})
    shap_local_df = shap_local_df.sort_values('abs_shap', ascending=False)
    shap_local_csv = os.path.join(CASE_DIR, f"case_{idx}_shap_local.csv")
    shap_local_df.to_csv(shap_local_csv, index=False)
    print("Saved SHAP local CSV for case", idx, "->", shap_local_csv)

    # Create a bar plot of top 10 shap contributions
    plt.figure(figsize=(8,4))
    topn = shap_local_df.head(10).sort_values('shap_value')
    plt.barh(topn['feature'], topn['shap_value'])
    plt.title(f"Case {idx} top SHAP contributions (signed)")
    plt.tight_layout()
    png_path = os.path.join(CASE_DIR, f"case_{idx}_shap_local.png")
    plt.savefig(png_path, dpi=150, bbox_inches='tight')
    plt.close()
    print("Saved SHAP local plot:", png_path)

    # LIME explanation (raw features)
    raw_vals = raw_row.values.ravel()
    exp = lime_explainer.explain_instance(raw_vals, predict_proba_raw, num_features=10)
    lime_list = exp.as_list()  # list of (feature, weight)
    lime_df = pd.DataFrame(lime_list, columns=['feature', 'weight'])
    lime_csv = os.path.join(CASE_DIR, f"case_{idx}_lime_local.csv")
    lime_df.to_csv(lime_csv, index=False)
    print("Saved LIME explanation for case", idx, "->", lime_csv)

# 15) Compare SHAP global vs model feature importance (XGBoost built-in)
# Get XGBoost feature importance (note: importance keys refer to preprocessed feature indices)
# We'll extract XGBoost feature importance on the preprocessed feature names:
bst = pipeline.named_steps['clf'].get_booster()
# XGBoost feature names default to f0, f1, ... when a numpy matrix is passed.
# We'll build a mapping: f{i} -> feature_names[i]
xgb_importance = pipeline.named_steps['clf'].get_booster().get_score(importance_type='gain')
# Convert to DataFrame
xgb_imp_list = []
for k,v in xgb_importance.items():
    # k like 'f12' -> index 12
    idx = int(k.replace('f',''))
    fname = feature_names[idx] if idx < len(feature_names) else f"f{idx}"
    xgb_imp_list.append((fname, v))
xgb_imp_df = pd.DataFrame(xgb_imp_list, columns=['feature','gain']).sort_values('gain', ascending=False)
XGB_IMP_CSV = "/mnt/data/xgb_feature_importance.csv"
xgb_imp_df.to_csv(XGB_IMP_CSV, index=False)
print("Saved XGBoost feature importance to:", XGB_IMP_CSV)

# 16) Save predictions on test set with SHAP top-3 features for each row (approx)
# Note: computing SHAP for entire test set may be slow; we'll compute for first 500 rows
test_N = min(500, X_test_preprocessed.shape[0])
shap_vals_full = explainer.shap_values(X_test_preprocessed[:test_N])
top3_list = []
for i in range(test_N):
    abs_vals = np.abs(shap_vals_full[i])
    top3_idx = np.argsort(abs_vals)[-3:][::-1]
    top3_features = [feature_names[j] for j in top3_idx]
    top3_list.append(";".join(top3_features))

pred_df = test_df.reset_index(drop=True).iloc[:test_N].copy()
pred_df['proba_default'] = pipeline.predict_proba(pred_df)[:,1]
pred_df['pred'] = pipeline.predict(pred_df)
pred_df['top3_shap'] = top3_list
OUT_PRED_CSV = "/mnt/data/test_predictions_with_top3_shap.csv"
pred_df.to_csv(OUT_PRED_CSV, index=False)
print("Saved sample test predictions with top3 SHAP to:", OUT_PRED_CSV)

# 17) Summary prints and file paths for deliverables
print("\n==== Deliverables (saved) ====")
print("Model pipeline:", MODEL_PATH)
print("SHAP summary plot:", SHAP_SUMMARY_PNG)
print("SHAP feature importance CSV:", SHAP_CSV)
print("XGBoost feature importance CSV:", XGB_IMP_CSV)
print("Per-case explanations folder:", CASE_DIR)
print("Sample test predictions with top3 SHAP:", OUT_PRED_CSV)

print("\nDone. You can download the saved files from the /mnt/data/ directory in the environment.")